In [1]:
from selenium.webdriver import ChromeOptions

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

import time

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

# = = =  = = =  = = =

import pandas as pd

In [2]:
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
# option.add_argument('--headless=new')
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument(get_header(method='selenium'))
option.add_argument(get_proxy(method='selenium'))
option.page_load_strategy = 'normal'

driver = Chrome(service=Service('../chromedriver'), options=option)
driver.maximize_window()

# = = =  = = =  = = =

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        driver.get('https://www.amazon.com/?language=en_US')
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
        time.sleep(0.7)
        driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
        time.sleep(1.2)
        driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")

        break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

driver

1


<selenium.webdriver.chrome.webdriver.WebDriver (session="1045c542c512b38850488855236f4e2e")>

In [3]:
list_url = []

list_url

[]

# 1. BSR

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree

import json

soup = BeautifulSoup(driver.page_source, 'lxml')
html = etree.HTML(str(soup))

list_dict = json.loads(html.xpath('//div[@data-client-recs-list]/@data-client-recs-list')[0])
list_url_temp = [f'''https://www.amazon.com/dp/{dict_['id']}''' for dict_ in list_dict]

len(list_url_temp), list_url_temp[-1]

In [ ]:
list_url.extend(list_url_temp)

len(list_url)

# 2. Search 1

In [4]:
from bs4 import BeautifulSoup
from lxml import etree

import json

soup = BeautifulSoup(driver.page_source, 'lxml')
html = etree.HTML(str(soup))

list_url_temp = [f'https://www.amazon.com{href.strip()}' for href in html.xpath('//div[@class="a-section a-spacing-none puis-padding-right-small s-title-instructions-style"]/h2/a/@href')]

len(list_url_temp), list_url_temp[-1]

IndexError: list index out of range

In [ ]:
list_url.extend(list_url_temp)

len(list_url)

# 3. Search 2

In [16]:
from bs4 import BeautifulSoup
from lxml import etree

import json

soup = BeautifulSoup(driver.page_source, 'lxml')
html = etree.HTML(str(soup))

list_url_temp = [f'https://www.amazon.com{href.strip()}' for href in html.xpath('//div[@class="a-section a-spacing-none a-spacing-top-small s-title-instructions-style"]/h2/a/@href')]

len(list_url_temp), list_url_temp[-1]

(24,
 'https://www.amazon.com/Kesoto-Conditioning-Refrigeration-Copper-Valve/dp/B07KMGRG3G/ref=sr_1_134?crid=31VO9AFIVUTQ7&dib=eyJ2IjoiMSJ9.ZtSydL7_OFRZTCHIiWcptswRRBwQO8r5OnBenEoldaQzWyOMAqZDJivP7-cSDHA1bo1w-DtqYtQRqOans2mWhsbJ1bSkLVEMybrcNI9LuPPbys1Pvay7aKjsG6tbYkbT0TM8vlctUPOhndrU5_gVoiNlyZuAVPpFZWwSmoc9lm5_IsmR_wvMQ1Tc8ujbtViqTSDEpI9YdT5ouCeIErMnxg.sx0KoVQvXrgE69Ogd0pWpvq6nX-gOoxRxGdn-fKH_Kg&dib_tag=se&keywords=ac+repair+kit&qid=1733466682&s=automotive&sprefix=ac+repair%2Cautomotive%2C968&sr=1-134')

In [17]:
list_url.extend(list_url_temp)

len(list_url)

153

# 4. temp

In [5]:
from bs4 import BeautifulSoup
from lxml import etree

import json

soup = BeautifulSoup(driver.page_source, 'lxml')
html = etree.HTML(str(soup))

list_url_temp = [f'https://www.amazon.com{href.strip()}' for href in html.xpath('//li[@data-testid="product-grid-item"]/div/a/@href')]

len(list_url_temp), list_url_temp[-1]

IndexError: list index out of range

In [ ]:
list_url.extend(list_url_temp)

len(list_url)

# = = =  = = =  = = =

In [ ]:
driver.quit()

print('Data Outputting...')
if list_url:
    df = pd.DataFrame({'No': range(1, len(list_url) + 1),
                       'Url': list_url})
    df.to_excel('./file/1.url.xlsx', index=False)
print('Output Finished !')
print()

df